In [1]:
import pandas as pd 
import numpy as np 
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
# 讀取資料 from 
train = pd.read_csv("C:/Users/User/Desktop/kaggle/Titanic/train.csv")
test  = pd.read_csv("C:/Users/User/Desktop//kaggle/Titanic//test.csv")
data = train.append(test)
data['Sex'] = data['Sex'].map ( { 'female':0 , 'male':1 } ).astype(int)

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [2]:
data['Fare'] = data['Fare'].map(lambda x: np.log10(x) if x>0 else 0)
data['Fare 4'] = pd.qcut(data['Fare'],4).astype(str)
data['Fare 5'] = pd.qcut(data['Fare'],5).astype(str)
data['Fare 6'] = pd.qcut(data['Fare'],6).astype(str)
#train['Fare 4'] = train['Fare 4'].astype(str)#轉換區間為字符以便encoder

label = LabelEncoder()
data['Fare 4'] = label.fit_transform(data['Fare 4'] )
data['Fare 5'] = label.fit_transform(data['Fare 5'] )
data['Fare 6'] = label.fit_transform(data['Fare 6'] )

In [3]:
MrNumpre = data[ (data['Name'].str.contains('Mr.') == True) & (data['Name'].str.contains('Mrs') == False)]
MrMedian_age = MrNumpre['Age'].median()

data.Age[ (data['Name'].str.contains('Mr.') == True) & (data['Name'].str.contains('Mrs') == False)
          & (data.Age.isnull()) ] = MrMedian_age

#adding Mrs
MrsNumpre = data[data['Name'].str.contains('Mrs.') == True]
MrsMedian_age = MrsNumpre['Age'].median()
data.Age[ (data['Name'].str.contains('Mrs') == True ) & (data.Age.isnull()) ] = MrsMedian_age

#adding Miss.
MissNumpre = data[data['Name'].str.contains('Miss.') == True]
MissMedian_age = MissNumpre['Age'].median()
data.Age[ (data['Name'].str.contains('Miss') == True ) & (data.Age.isnull()) ] = MissMedian_age

#adding Master
MasterNumpre = data[data['Name'].str.contains('Master') == True]
MasterMedian_age = MasterNumpre['Age'].median()
data.Age[ (data['Name'].str.contains('Master') == True ) & (data.Age.isnull()) ] = MasterMedian_age

#adding Dr.
DrNumpre = data[data['Name'].str.contains('Dr.') == True]
DrMedian_age = DrNumpre['Age'].median()
data.Age[ (data['Name'].str.contains('Dr') == True ) & (data.Age.isnull()) ] = DrMedian_age

data.Age[data.Age.isnull()] = 25

data['Age3'] = data['Age'] 
data.Age3[data['Age'] < 15 ] = 1
data.Age3[ data['Age'].between(15,30)] = 2
data.Age3[data['Age'] > 30 ] = 2

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_la

In [4]:
Age=['Sex','Pclass','Fare 5','Age3']
test  = data[data['Survived'].isnull()]
train = data[data['Survived'].notnull()]

Y = train['Survived']
X = train.drop(['Survived','PassengerId'],axis=1)

Model = RandomForestClassifier(random_state=2,n_estimators=250,min_samples_split=20,oob_score=True)
Model.fit(X[Age],Y)
print(Model.oob_score_)

0.8114478114478114


In [5]:
X_submit = test.drop(['PassengerId'],axis=1)
Y_submit = Model.predict(X_submit[Age])

submit = pd.DataFrame({"PassengerId": test['PassengerId'],
                      "Survived":Y_submit.astype(int)})
submit.to_csv("submit Age.csv",index=False)